In [379]:
import http.client
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests
import config

In [380]:
url1 = f"https://api.sportmonks.com/v3/football/fixtures/date/2022-09-06?api_token={config.api_key}&include=scores;participants"
url2 = f"https://api.sportmonks.com/v3/football/fixtures/date/2022-09-03?api_token={config.api_key}&include=scores;participants"

response2 = requests.get(url2)
data2 = response2.json()
print('data' in data2)

response1 = requests.get(url1)
data1 = response1.json()
print('data' in data1)



True
False


In [381]:
url = f"https://api.sportmonks.com/v3/football/fixtures/date/2022-09-03?api_token={config.api_key}&include=scores;participants"

response = requests.get(url)
data = response.json()
df = pd.DataFrame(data['data'])\
       .drop(columns=['id', 'sport_id', 'stage_id', 'league_id', 'season_id', 'group_id', 'aggregate_id', 'round_id', 'state_id', 'venue_id', 'leg','details','length','placeholder','has_odds','starting_at_timestamp'])

In [382]:
df['Date'] = pd.to_datetime(df['starting_at']).dt.date
df = df.rename(columns={'name':'Game Match'})


In [383]:
scores = pd.json_normalize(df.scores, sep='_').rename(columns={4:'home_score', 5:'away_score'}).drop(columns=[0,1,2,3])
home_scores = pd.json_normalize(scores.home_score, sep='_')\
                .rename(columns={'score_goals':'total_home_goals'})\
                .drop(columns=['score_participant', 'description', 'type_id', 'fixture_id', 'id', 'participant_id'])

away_scores = pd.json_normalize(scores.away_score, sep='_')\
                .rename(columns={'score_goals':'total_away_goals'})\
                .drop(columns=['score_participant', 'description', 'type_id', 'fixture_id', 'id', 'participant_id'])



In [384]:

participants = pd.json_normalize(df.participants, sep='_').rename(columns={0:'home', 1:'away'})

away = pd.json_normalize(participants.away, sep='_')\
         .rename(columns={'id':'away_id', 'name':'away_name', 'image_path':'away_image_path', 'founded':'away_founded', 'short_code': 'away_short_code', 'country_id': 'away_country_id'})\
         .drop(columns=['sport_id', 'type', 'placeholder', 'meta_location', 'meta_winner', 'meta_position', 'gender', 'last_played_at', 'venue_id'])

home = pd.json_normalize(participants.home, sep='_')\
         .rename(columns={'id':'home_id', 'name':'home_name', 'image_path':'home_image_path', 'founded':'home_founded', 'short_code': 'home_short_code', 'country_id': 'home_country_id'})\
         .drop(columns=['sport_id', 'type', 'placeholder', 'meta_location', 'meta_winner', 'meta_position', 'gender', 'last_played_at', 'venue_id'])

df = pd.concat([df, home, away, home_scores, away_scores], axis=1)

In [385]:
df_new = df.drop(columns=['scores', 'participants', 'starting_at'])
df_new['home_win'] = np.where(df_new['total_home_goals'] > df_new['total_away_goals'], 1, 0)
df_new['away_win'] = np.where(df_new['total_home_goals'] < df_new['total_away_goals'], 1, 0)
df_new = df_new[['Date', 'Game Match', 'result_info', 'home_id', 'home_name', 'home_image_path', 'home_founded', 'home_short_code', 'home_country_id', 'total_home_goals', 'home_win', 'away_id', 'away_name', 'away_image_path', 'away_founded', 'away_short_code', 'away_country_id', 'total_away_goals', 'away_win']]

In [386]:
df_new


,Date,Game Match,result_info,home_id,home_name,home_image_path,home_founded,home_short_code,home_country_id,total_home_goals,home_win,away_id,away_name,away_image_path,away_founded,away_short_code,away_country_id,total_away_goals,away_win
0,2022-09-03,Celtic vs Rangers,Celtic won after full-time.,53,Celtic,https://cdn.sportmonks.com/images/soccer/teams...,1888,CEL,1161,4,1,62,Rangers,https://cdn.sportmonks.com/images/soccer/teams...,1873,RAN,1161,0,0
1,2022-09-03,Hibernian vs Kilmarnock,Hibernian won after full-time.,66,Hibernian,https://cdn.sportmonks.com/images/soccer/teams...,1875,HIB,1161,1,1,180,Kilmarnock,https://cdn.sportmonks.com/images/soccer/teams...,1869,KIL,1161,0,0
2,2022-09-03,Livingston vs Hearts,Livingston won after full-time.,258,Livingston,https://cdn.sportmonks.com/images/soccer/teams...,1943,LIV,1161,1,1,314,Hearts,https://cdn.sportmonks.com/images/soccer/teams...,1874,None,1161,0,0
3,2022-09-03,Motherwell vs Dundee United,Game ended in draw.,309,Motherwell,https://cdn.sportmonks.com/images/soccer/teams...,1886,MOT,1161,0,0,282,Dundee United,https://cdn.sportmonks.com/images/soccer/teams...,1909,DUD,1161,0,0
4,2022-09-03,Ross County vs Aberdeen,Game ended in draw.,273,Aberdeen,https://cdn.sportmonks.com/images/soccer/teams...,1903,ABE,1161,1,0,246,Ross County,https://cdn.sportmonks.com/images/soccer/teams...,1929,RSC,1161,1,0
5,2022-09-03,St. Johnstone vs St. Mirren,St. Johnstone won after full-time.,734,St. Johnstone,https://cdn.sportmonks.com/images/soccer/teams...,1885,STJ,1161,3,1,496,St. Mirren,https://cdn.sportmonks.com/images/soccer/teams...,1877,STM,1161,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              6 non-null      object
 1   Game Match        6 non-null      object
 2   result_info       6 non-null      object
 3   home_id           6 non-null      int64 
 4   home_name         6 non-null      object
 5   home_image_path   6 non-null      object
 6   home_founded      6 non-null      int64 
 7   home_short_code   6 non-null      object
 8   home_country_id   6 non-null      int64 
 9   total_home_goals  6 non-null      int64 
 10  home_win          6 non-null      int64 
 11  away_id           6 non-null      int64 
 12  away_name         6 non-null      object
 13  away_image_path   6 non-null      object
 14  away_founded      6 non-null      int64 
 15  away_short_code   5 non-null      object
 16  away_country_id   6 non-null      int64 
 17  total_away_goals  6 